In [2]:
import pandas as pd
import json
from openai import OpenAI
client=OpenAI()

## Ingestion

In [3]:
df=pd.read_csv('../data/update_category.csv')
df.head()

,id,Question,Answer,Category
0,0,What are some examples of breed-typical person...,Breed-typical personalities in dogs are develo...,{'Dog'}
1,1,How do working and herding dogs typically behave?,Working and herding dogs have business-like di...,{'Dog'}
2,2,Which breed of dogs is known for strong loyalty?,Collies and Akitas are known for their strong ...,{'Dog'}
3,3,What instincts do guarding dogs typically disp...,Guarding dogs tend to be protective of their t...,{'Dog'}
4,4,How does breed specificity impact a dog's abil...,Breed specificity affects how well dogs adapt ...,{'Dog'}


In [4]:
documents = df.to_dict(orient='records')

# data = df.to_dict(orient='records')


In [5]:
documents[0]

{'id': 0,
 'Question': 'What are some examples of breed-typical personalities in dogs?',
 'Answer': 'Breed-typical personalities in dogs are developed through generations of selection for certain traits. For example, sporting dogs are adventurous and responsive to humans, while hounds tend to be more independent and aloof.',
 'Category': "{'Dog'}"}

In [6]:
prompt_template = """
You are simulating a pet owner who's seeking advice about their dog or cat.
Formulate 5 questions this pet owner might ask based on a FAQ record.
The record should contain the answer to the questions, and the questions should be complete 
and not too short.Use as fewer words as possible from the record.

The record:

question: {Question}
answer: {Answer}
category: {Category}

Provide the output in parsable JSON:
{{"questions": ["question1", "question2",..., "question5"]}}
""".strip()


In [7]:

prompt = prompt_template.format(**documents[0])
prompt

'You are simulating a pet owner who\'s seeking advice about their dog or cat.\nFormulate 5 questions this pet owner might ask based on a FAQ record.\nThe record should contain the answer to the questions, and the questions should be complete \nand not too short.Use as fewer words as possible from the record.\n\nThe record:\n\nquestion: What are some examples of breed-typical personalities in dogs?\nanswer: Breed-typical personalities in dogs are developed through generations of selection for certain traits. For example, sporting dogs are adventurous and responsive to humans, while hounds tend to be more independent and aloof.\ncategory: {\'Dog\'}\n\nProvide the output in parsable JSON:\n{"questions": ["question1", "question2",..., "question5"]}'

In [8]:
# def generate_questions(doc):
#     prompt = prompt_template.format(**doc)

#     response = client.chat.completions.create(
#         model='gpt-4o-mini',
#         messages=[{"role": "user", "content": prompt}]
#     )

#     json_response = response.choices[0].message.content
#     return json_response

In [9]:
# json_response=generate_questions(documents[0])

NameError: name 'generate_questions' is not defined

In [ ]:
# json_response

'{"questions": ["What types of personalities are common in dogs based on their breed?", "How do breed-specific traits influence dog behavior?", "Can you give examples of sporting dog personalities?", "What personality traits are typical of hound breeds?", "How does selection for certain traits affect a dog\'s personality?"]}'

In [10]:
from tqdm.auto import tqdm


In [13]:
results = {}


In [12]:
len(documents)
# documents[28]

583

  0%|          | 0/583 [00:00<?, ?it/s]

In [1]:
len(results)

NameError: name 'results' is not defined

In [14]:
import pickle


In [29]:
with open('questions.pkl', 'wb') as f:
    pickle.dump(results, f)

In [18]:
import json
import time
from tqdm import tqdm
from functools import wraps
import threading

class TimeoutError(Exception):
    pass

def timeout(seconds):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            result = [TimeoutError('function [%s] timeout [%s seconds]' % (func.__name__, seconds))]
            def worker():
                try:
                    result[0] = func(*args, **kwargs)
                except Exception as e:
                    result[0] = e
            t = threading.Thread(target=worker)
            t.daemon = True
            try:
                t.start()
                t.join(seconds)
            except Exception as e:
                print('error starting thread')
                raise e
            ret = result[0]
            if isinstance(ret, BaseException):
                raise ret
            return ret
        return wrapper
    return decorator

@timeout(30)  # 設置 30 秒超時
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response


def save_results(results):
    with open('results.json', 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

def process_documents(documents, results, save_interval=10):
    start_time = time.time()
    for i, doc in enumerate(tqdm(documents)):
        doc_id = doc['id']
        if doc_id in results:
            continue

        try:
            questions_raw = generate_questions(doc)
            questions = json.loads(questions_raw)
            results[doc_id] = questions['questions']
        except TimeoutError:
            print(f"處理文檔 {doc_id} 時超時，跳過該文檔")
            continue
        except json.JSONDecodeError:
            print(f"解析文檔 {doc_id} 的問題時出錯")
            print(questions_raw)
            print(doc)
            break
        except Exception as e:
            print(f"處理文檔 {doc_id} 時發生錯誤: {str(e)}")
            break

        # 每處理 save_interval 個文檔就保存一次結果
        if (i + 1) % save_interval == 0:
            save_results(results)
            print(f"已保存 {i + 1} 個文檔的結果")

        # 檢查總運行時間
        if time.time() - start_time > 3600:  # 1小時後停止
            print("程序運行超過1小時，停止處理")
            break

    # 最後再保存一次結果
    save_results(results)
    print("處理完成，已保存所有結果")

# 使用示例


In [27]:

# documents = [...]  # 您的文檔列表
# results = {}  # 如果有之前的結果，可以從文件中加載

process_documents(documents, results)

 56%|█████▌    | 325/583 [00:01<00:00, 268.50it/s]

已保存 330 個文檔的結果


 57%|█████▋    | 334/583 [00:12<00:13, 18.71it/s] 

已保存 340 個文檔的結果


 60%|█████▉    | 347/583 [00:30<00:44,  5.30it/s]

...

已保存 570 個文檔的結果


 99%|█████████▉| 580/583 [04:59<00:03,  1.07s/it]

已保存 580 個文檔的結果


100%|██████████| 583/583 [05:02<00:00,  1.93it/s]

處理完成，已保存所有結果


In [ ]:
# for doc in tqdm(documents): 
#     doc_id = doc['id']
#     if doc_id in results:
#         continue

#     questions_raw = generate_questions(doc)
#     try:
#         questions=json.loads(questions_raw)    
#         results[doc_id] = questions['questions']
#     except:
#         print(questions_raw)
#         print(doc)
#         break    

In [30]:
len(documents),len(results)

(583, 583)

In [32]:
# results

In [16]:

# 使用示例
# documents = [...]  # 您的文檔列表
# results #= {}  # 如果有之前的結果，可以從文件中加載

# process_documents(documents, results)

 33%|███▎      | 190/583 [00:00<?, ?it/s]

處理文檔 190 時發生錯誤: module 'signal' has no attribute 'SIGALRM'
處理完成，已保存所有結果


In [11]:
# import pickle

# # 從檔案讀取 dictionary
# with open('questions.pkl', 'rb') as file:
#     loaded_dict = pickle.load(file)

# print(loaded_dict)
# results=loaded_dict

{0: ['What characteristics define sporting dog personalities?', 'How do hounds typically behave in terms of personality?', 'What influences the personalities of different dog breeds?', 'In what ways do breed-specific traits affect dog behavior?', 'Can you explain the typical temperaments of sporting dogs compared to hounds?'], 1: ['What is the typical behavior of working and herding dogs?', 'How do herding dogs evaluate situations?', 'Do Collie puppies show any instinctive behaviors?', 'What kind of disposition do working dogs have?', 'How do working dogs approach their tasks?'], 2: ['What dog breeds are recognized for their loyalty?', 'Can you name a loyal dog breed?', 'Which breeds exemplify loyalty in dogs?', 'What are some examples of loyal dog breeds?', 'Are Collies and Akitas considered loyal dogs?'], 3: ['What natural behaviors are common in young guarding dogs?', "How do early experiences influence a guarding dog's protective instincts?", 'What are guarding dogs naturally prote

In [33]:
final_results=[]
for doc_id, questions in results.items():
    for q in questions:
        final_results.append((doc_id, q))
      

In [34]:
final_results[0]

(0, 'What characteristics define sporting dog personalities?')

In [35]:
df_results=pd.DataFrame(final_results,columns=['id','question'])

In [36]:
df_results.to_csv('../data/ground-truth-retrieval.csv',index=False)

In [40]:
df_retrival=pd.read_csv("../data/ground-truth-retrieval.csv",index_col=0)

In [41]:
df_retrival

,question
id,
0,What characteristics define sporting dog perso...
0,How do hounds typically behave in terms of per...
0,What influences the personalities of different...
0,In what ways do breed-specific traits affect d...
0,Can you explain the typical temperaments of sp...
...,...
582,What are effective methods for addressing sepa...
582,How can I make my pet feel better when I leave...
582,What role does crate training play in helping ...
